In [ ]:
# https://ai.meta.com/vjepa/

In [ ]:
# ==========================================================
# 1️⃣ 라이브러리 설치 및 환경 설정
# ==========================================================

# Hugging Face Transformers, Torchcodec, Decord 설치
# Colab 환경에서는 !pip 사용 가능
!pip install transformers==4.52.4
!pip install torchcodec==0.2.1
!pip install decord

# PyTorch 설치 (GPU 사용 환경)
!pip install torch==2.6.0 torchvision==0.17.0 torchaudio==2.6.0

ERROR: Could not find a version that satisfies the requirement torchcodec==0.2.1 (from versions: 0.0.0.dev0, 0.7.0, 0.8.0, 0.8.1)
ERROR: No matching distribution found for torchcodec==0.2.1


   ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
   -- ------------------------------------- 1.6/24.7 MB 11.3 MB/s eta 0:00:03
   ------------------------ --------------- 15.2/24.7 MB 46.2 MB/s eta 0:00:01
   -------------------------- ------------- 16.3/24.7 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------  24.6/24.7 MB 34.0 MB/s eta 0:00:01
   ---------------------------------------- 24.7/24.7 MB 29.0 MB/s  0:00:00
  Using cached regex-2025.10.23-cp313-cp313-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 10.5/10.5 MB 57.9 MB/s  0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 23.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 61.8 MB/s  0:00:00
Using cached regex-2025.10

ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3
ERROR: Could not find a version that satisfies the requirement torchvision==0.17.0 (from versions: 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.24.0)
ERROR: No matching distribution found for torchvision==0.17.0
c:\Users\Playdata2\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ==========================================================
# 2️⃣ 모델과 비디오 프로세서 로드
# ==========================================================

from transformers import AutoVideoProcessor, AutoModel
import torch

# 사전학습된 V-JEPA2 모델 이름
hf_repo = "facebook/vjepa2-vitl-fpc64-256"

# 모델 로드 (GPU 사용)
model = AutoModel.from_pretrained(hf_repo).to("cuda")  # GPU가 없으면 "cpu"로 변경
# 비디오 전처리용 processor 로드
processor = AutoVideoProcessor.from_pretrained(hf_repo)

In [ ]:
# ==========================================================
# 3️⃣ 테스트용 비디오 불러오기
# ==========================================================
import numpy as np
import decord
from decord import VideoReader, cpu

# 테스트용 비디오 파일 경로
video_path = "test_video.mp4"  # 자신의 비디오 파일 경로로 변경

# 비디오 로드
vr = VideoReader(video_path, ctx=cpu(0))
video = vr[:].asnumpy()  # 전체 프레임을 numpy array로 변환 (shape: num_frames x H x W x 3)

print(f"비디오 프레임 개수: {video.shape[0]}")
print(f"비디오 해상도: {video.shape[1]}x{video.shape[2]}")

In [ ]:
# ==========================================================
# 4️⃣ 프레임 선택 및 전처리
# ==========================================================

# 모델 입력용으로 64개 프레임만 선택
num_frames = 64
frame_idx = np.linspace(0, video.shape[0]-1, num_frames).astype(int)  # 0~마지막 프레임 사이 균등 선택
video_frames = video[frame_idx]

# processor를 통해 전처리
inputs = processor(video_frames, return_tensors="pt").to("cuda")  # GPU 사용

print("입력 전처리 완료")
print(f"입력 텐서 shape: {inputs['pixel_values'].shape}")  # (1, 64, 3, H, W)

In [ ]:
# ==========================================================
# 5️⃣ 모델로 비디오 특징 추출
# ==========================================================

with torch.no_grad():  # 추론 시 gradient 계산 불필요
    features = model.get_vision_features(**inputs)  # shape: (1, 64, 1024) 예시

print("모델 특징 추출 완료")
print(f"추출된 특징 shape: {features.shape}")

In [ ]:
# ==========================================================
# 6️⃣ 다운스트림 활용 예시
# ==========================================================

# features를 사용하여 영상 검색, 분류, 클러스터링 등 가능
# 예시: 각 프레임 평균 임베딩 계산
video_embedding = features.mean(dim=1)
print(f"영상 전체 임베딩 shape: {video_embedding.shape}")  # (1, 1024)
